In [230]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB2 --baud=115200 
Ready.


In [81]:
%sendtofile config.txt

wifiname     DoESLiverpool
wifipassword decafbad00
mqttbroker   10.0.30.130
pinled       2
boardname    esp8266LOLIN

Sent 5 lines (116 bytes) to config.txt.


In [229]:
%sendtofile main.py

# Very minimal scrolling panel
numledcells = const(4*4)
mstimedelay = const(20)

# setup and initialize the LED strip
from machine import Pin, SPI
spi = SPI(1, baudrate=10000000, polarity=0, phase=0)
cs = Pin(15, Pin.OUT, value=1)
for i in range(0, 10, 2):
    cs.value(0)
    x = b"\x0c\x00\x0f\x00\x0b\x07\x09\x00\x0c\x01"[i:i+2]
    spi.write(x*numledcells)
    cs.value(1)
    
cs.value(0)  # 0<=brightness<=15
spi.write(bytes([10, 0])*numledcells)
cs.value(1)

import framebuf
buffer = bytearray(8*(numledcells+1))  # make one extra character to scroll from
fbuff = framebuf.FrameBuffer(buffer, 8*(numledcells+1), 8, framebuf.MONO_HLSB)


def show():
    for y in range(8):
        cs.value(0)
        for m in range(numledcells):
            spi.write(bytearray([1 + y, buffer[y*(numledcells+1) + (numledcells - m - 1)]]))
        cs.value(1)

for i in range(0, numledcells*8, 4):
    fbuff.fill_rect(i, (i%8), 4, 4, 1)
show()

import time
message = "DoESLiverpool is...?"
messagechar = -1
messagepix = 8
def msgnext():
    global messagechar, messagepix
    messagepix += 1
    if messagepix < 8:
        fbuff.scroll(-1,0)
    else:
        messagechar += 1
        messagepix = 0
        fbuff.fill(0)
        mc = max(messagechar, 0)
        fbuff.text(message[mc:mc+numledcells+1], -min(0, messagechar)*8, 0, 1)
    show()

#while 1:
#    msgnext()
#    if messagechar >= len(message):
#        messagechar, messagepix = -numledcells, 7
#    time.sleep_ms(20)


# mqtt technology
import network, machine, ubinascii

fconfig = dict(x.split()  for x in open("config.txt"))
pled = machine.Pin(int(fconfig["pinled"]), machine.Pin.OUT)

si = network.WLAN()
si.active(True)
si.connect(fconfig["wifiname"], fconfig["wifipassword"])
while not si.isconnected():
    time.sleep_ms(500)
ipnumber = si.ifconfig()[0]
print(ipnumber)
for i in range(10):
    pled.value(i%2)
    time.sleep_ms(100)

from umqtt.robust import MQTTClient
client = MQTTClient(fconfig["boardname"], fconfig["mqttbroker"])
client.connect()
client.publish(fconfig["boardname"]+"/ip", ipnumber, retain=True, qos=1)
client.publish(fconfig["boardname"]+"/mac", ubinascii.hexlify(si.config('mac'),':').decode().upper(), retain=True, qos=1)


def callback(topic, msg):
    global message, messagechar, messagepix
    print((topic, msg))
    message = msg
    messagechar, messagepix = -numledcells, 7
client.set_callback(callback)
client.subscribe(fconfig["boardname"]+"/message")

tnext = 0
while True:
    while time.ticks_ms() < tnext:
        pass
    tnext = time.ticks_ms() + mstimedelay

    msgnext()
    if messagechar >= len(message):
        messagechar, messagepix = -numledcells, 7
    client.check_msg()


Sent 95 lines (2572 bytes) to main.py.
